# Fast-Fourier-Transformation Example

## Machine-Learned Inverse FFT

Let's start with a sinusoidal curve: that's the input (`x`) that we want to learn, given its FFT (`y`).
The goal is to do an `InverseRealFFT` by gradient descent.

For this problem the input (`x`) is real (`float32`) and label (`y`), the FFT, is complex (`complex64`).

In [1]:
!*rm -f go.work && go work init && go work use . "${HOME}/Projects/gomlx"
%goworkfix

	- Added replace rule for module "github.com/gomlx/gomlx" to local directory "/home/janpf/Projects/gomlx".


In [2]:
import (
    "github.com/gomlx/gopjrt/dtypes"
    . "github.com/gomlx/gomlx/pkg/core/graph"
    . "github.com/gomlx/exceptions"
    "github.com/gomlx/gomlx/pkg/core/tensors"
    "github.com/gomlx/gomlx/pkg/support/xslices"
    mg "github.com/gomlx/gomlx/ui/gonb/margaid"
    "github.com/janpfeifer/gonb/gonbui"

    _ "github.com/gomlx/gomlx/backends/default"
)

// manager always created at initialization.
var backend = backends.MustNew()

const (
    NumPoints = 100
    Frequency = 2.0  // Number of curves in samples.
    RealDType = dtypes.Float32
    ComplexDType = dtypes.Complex64
)

// CalculateXY returns (x, y) of our problem, where y is a sinusoidal curve and x is its FFT.
func CalculateXY() (x, y *tensors.Tensor) {
    e := MustNewExec(backend, func (g *Graph) (x, y *Node) {
        x = Iota(g, shapes.Make(RealDType, 1, NumPoints), 1)
        x = MulScalar(x, 2.0*math.Pi*Frequency/float64(NumPoints))
        x = Sin(x)
        y = RealFFT(x)
        return
    })
    return e.MustExec2()
}

func Plot(displayId string, width, height int, coordinates []*tensors.Tensor, names []string) {
    plts := mg.New(width, height)
    for ii, t := range coordinates {
        var values []float64
        switch t.DType() {
        case dtypes.Float64:
            values = tensors.CopyFlatData[float64](t)
        case dtypes.Float32:
            values32 := tensors.CopyFlatData[float32](t)
            values = xslices.Map(values32, func (v float32) float64 { return float64(v) })
        default:
            Panicf("only float32 and float64 tensor dtypes are accepted by Plot, got t.shape=%s", t.Shape())
        }
        var name string
        if len(names) > ii {
            name = names[ii]
        }
        plts.AddValues(name, "", values)
    }
    if displayId == "" {
        plts.Plot()
    } else {
        gonbui.UpdateHTML(displayId, plts.PlotToHTML())
    }
}

%%
x, y := CalculateXY()
fmt.Printf("x: shape=%s\n", x.Shape())
fmt.Printf("y: shape=%s\n", y.Shape())
Plot("", 1024, 320, []*tensors.Tensor{x}, nil)

x: shape=(Float32)[1 100]
y: shape=(Complex64)[1 51]


<path vector-effect="non-scaling-stroke" d="M1.768000e+01,9.000000e+01 L2.625212e+01,1.008502e+02 L3.482424e+01,1.115293e+02 L4.339636e+01,1.218689e+02 L5.196848e+01,1.317058e+02 L6.054061e+01,1.408851e+02 L6.911273e+01,1.492618e+02 L7.768485e+01,1.567040e+02 L8.625697e+01,1.630942e+02 L9.482909e+01,1.683316e+02 L1.034012e+02,1.723338e+02 L1.119733e+02,1.750374e+02 L1.205455e+02,1.764000e+02 L1.291176e+02,1.764000e+02 L1.376897e+02,1.750374e+02 L1.462618e+02,1.723337e+02 L1.548339e+02,1.683316e+02 L1.634061e+02,1.630942e+02 L1.719782e+02,1.567040e+02 L1.805503e+02,1.492618e+02 L1.891224e+02,1.408851e+02 L1.976945e+02,1.317058e+02 L2.062667e+02,1.218688e+02 L2.148388e+02,1.115293e+02 L2.234109e+02,1.008502e+02 L2.319830e+02,9.000000e+01 L2.405552e+02,7.914979e+01 L2.491273e+02,6.847070e+01 L2.576994e+02,5.813114e+01 L2.662715e+02,4.829417e+01 L2.748436e+02,3.911494e+01 L2.834158e+02,3.073819e+01 L2.919879e+02,2.329603e+01 L3.005600e+02,1.690582e+01 L3.091321e+02,1.166837e+01 L3.177042e+02,7.666243e+00 L3.262764e+02,4.962576e+00 L3.348485e+02,3.600005e+00 L3.434206e+02,3.600005e+00 L3.519927e+02,4.962587e+00 L3.605648e+02,7.666258e+00 L3.691370e+02,1.166838e+01 L3.777091e+02,1.690585e+01 L3.862812e+02,2.329605e+01 L3.948533e+02,3.073821e+01 L4.034255e+02,3.911497e+01 L4.119976e+02,4.829422e+01 L4.205697e+02,5.813120e+01 L4.291418e+02,6.847076e+01 L4.377139e+02,7.914981e+01 L4.462861e+02,9.000002e+01 L4.548582e+02,1.008502e+02 L4.634303e+02,1.115293e+02 L4.720024e+02,1.218689e+02 L4.805745e+02,1.317058e+02 L4.891467e+02,1.408851e+02 L4.977188e+02,1.492618e+02 L5.062909e+02,1.567040e+02 L5.148630e+02,1.630942e+02 L5.234352e+02,1.683316e+02 L5.320073e+02,1.723338e+02 L5.405794e+02,1.750374e+02 L5.491515e+02,1.764000e+02 L5.577236e+02,1.764000e+02 L5.662958e+02,1.750374e+02 L5.748679e+02,1.723337e+02 L5.834400e+02,1.683316e+02 L5.920121e+02,1.630941e+02 L6.005842e+02,1.567039e+02 L6.091564e+02,1.492618e+02 L6.177285e+02,1.408850e+02 L6.263006e+02,1.317058e+02 L6.348727e+02,1.218688e+02 L6.434448e+02,1.115292e+02 L6.520170e+02,1.008501e+02 L6.605891e+02,9.000000e+01 L6.691612e+02,7.914979e+01 L6.777333e+02,6.847070e+01 L6.863055e+02,5.813114e+01 L6.948776e+02,4.829417e+01 L7.034497e+02,3.911493e+01 L7.120218e+02,3.073817e+01 L7.205939e+02,2.329602e+01 L7.291661e+02,1.690582e+01 L7.377382e+02,1.166836e+01 L7.463103e+02,7.666243e+00 L7.548824e+02,4.962576e+00 L7.634545e+02,3.600000e+00 L7.720267e+02,3.600010e+00 L7.805988e+02,4.962597e+00 L7.891709e+02,7.666274e+00 L7.977430e+02,1.166841e+01 L8.063152e+02,1.690588e+01 L8.148873e+02,2.329609e+01 L8.234594e+02,3.073825e+01 L8.320315e+02,3.911502e+01 L8.406036e+02,4.829427e+01 L8.491758e+02,5.813117e+01 L8.577479e+02,6.847074e+01 L8.663200e+02,7.914983e+01 "/> 0 10 20 30 40 50 60 70 80 90 Steps -0.000

### Train the model

If you run it, you'll see the plot of the "learnedX" adjusting towards "x", the original sinusoidal curve.

In [3]:
import (
    . "github.com/gomlx/gomlx/pkg/core/graph"
    "github.com/gomlx/gomlx/pkg/ml/context"
    "github.com/gomlx/gomlx/pkg/ml/data"
    "github.com/gomlx/gomlx/pkg/ml/train"
)

var (
	flagNumSteps     = flag.Int("steps", 1000, "Number of gradient descent steps to perform")
	flagLearningRate = flag.Float64("learning_rate", 0.1, "Initial learning rate.")
)

func TrainInverseRealFFT() {
    x, y := CalculateXY()
    ctx := context.New()
	ctx.SetParam(optimizers.LearningRateKey, *flagLearningRate)

    modelFn := func(ctx *context.Context, spec any, inputs []*Node) []*Node {
        g := inputs[0].Graph()
        learnedXVar := ctx.VariableWithShape("learnedX", x.Shape())
        predictedY := RealFFT(learnedXVar.ValueGraph(g))
        return []*Node{predictedY}
    }

    dataset, err := data.InMemoryFromData(backend, "dataset", []any{x}, []any{y})
    if err != nil {
        panic(err)
    }
    dataset.BatchSize(1, false).Infinite(true)

    opt := optimizers.Adam().Done()
    trainer := train.NewTrainer(
        backend, ctx, modelFn,
        losses.MeanAbsoluteError,
        opt,
        nil, nil) // trainMetrics, evalMetrics

	loop := train.NewLoop(trainer)
	commandline.AttachProgressBar(loop) // Attaches a progress bar to the loop.

    // Plot learnedX
    displayId := gonbui.UniqueID()
    gonbui.UpdateHTML(displayId, "")
    train.EveryNSteps(loop, 10, "plot", 0, func(loop *train.Loop, metrics []*tensors.Tensor) error {
        learnedXVar := ctx.InspectVariable(context.RootScope, "learnedX")
        learnedX := learnedXVar.Value()
        Plot(displayId, 1024, 320, []*tensors.Tensor{x, learnedX}, []string{"Truth", "Learned"})
        return nil
    })
    
	// Loop for given number of steps.
	_, err = loop.RunSteps(dataset, *flagNumSteps)
	if err != nil {
		panic(err)
	}

}

%% --steps=800 --learning_rate=0.01
TrainInverseRealFFT()

<path vector-effect="non-scaling-stroke" d="M1.768000e+01,9.018442e+01 L2.625212e+01,1.009365e+02 L3.482424e+01,1.116592e+02 L4.339636e+01,1.218885e+02 L5.196848e+01,1.317009e+02 L6.054061e+01,1.406720e+02 L6.911273e+01,1.492110e+02 L7.768485e+01,1.566179e+02 L8.625697e+01,1.630492e+02 L9.482909e+01,1.681726e+02 L1.034012e+02,1.722598e+02 L1.119733e+02,1.748783e+02 L1.205455e+02,1.764000e+02 L1.291176e+02,1.761877e+02 L1.376897e+02,1.748893e+02 L1.462618e+02,1.722069e+02 L1.548339e+02,1.681859e+02 L1.634061e+02,1.630325e+02 L1.719782e+02,1.566730e+02 L1.805503e+02,1.491776e+02 L1.891224e+02,1.408337e+02 L1.976945e+02,1.316888e+02 L2.062667e+02,1.218477e+02 L2.148388e+02,1.114828e+02 L2.234109e+02,1.008871e+02 L2.319830e+02,9.001868e+01 L2.405552e+02,7.924377e+01 L2.491273e+02,6.854082e+01 L2.576994e+02,5.829190e+01 L2.662715e+02,4.848711e+01 L2.748436e+02,3.921227e+01 L2.834158e+02,3.083985e+01 L2.919879e+02,2.350122e+01 L3.005600e+02,1.702543e+01 L3.091321e+02,1.182844e+01 L3.177042e+02,7.997745e+00 L3.262764e+02,5.378169e+00 L3.348485e+02,3.842576e+00 L3.434206e+02,3.821945e+00 L3.519927e+02,5.160807e+00 L3.605648e+02,7.844087e+00 L3.691370e+02,1.189244e+01 L3.777091e+02,1.690206e+01 L3.862812e+02,2.340879e+01 L3.948533e+02,3.106863e+01 L4.034255e+02,3.937168e+01 L4.119976e+02,4.837920e+01 L4.205697e+02,5.842303e+01 L4.291418e+02,6.861834e+01 L4.377139e+02,7.899020e+01 L4.462861e+02,9.007014e+01 L4.548582e+02,1.008416e+02 L4.634303e+02,1.114966e+02 L4.720024e+02,1.218146e+02 L4.805745e+02,1.315460e+02 L4.891467e+02,1.408036e+02 L4.977188e+02,1.493727e+02 L5.062909e+02,1.566447e+02 L5.148630e+02,1.630798e+02 L5.234352e+02,1.682840e+02 L5.320073e+02,1.719686e+02 L5.405794e+02,1.748051e+02 L5.491515e+02,1.763590e+02 L5.577236e+02,1.763492e+02 L5.662958e+02,1.750504e+02 L5.748679e+02,1.721647e+02 L5.834400e+02,1.680819e+02 L5.920121e+02,1.628052e+02 L6.005842e+02,1.567074e+02 L6.091564e+02,1.488831e+02 L6.177285e+02,1.407857e+02 L6.263006e+02,1.318675e+02 L6.348727e+02,1.218919e+02 L6.434448e+02,1.114634e+02 L6.520170e+02,1.009062e+02 L6.605891e+02,8.992699e+01 L6.691612e+02,7.926046e+01 L6.777333e+02,6.861514e+01 L6.863055e+02,5.802950e+01 L6.948776e+02,4.830800e+01 L7.034497e+02,3.944058e+01 L7.120218e+02,3.072777e+01 L7.205939e+02,2.353566e+01 L7.291661e+02,1.703110e+01 L7.377382e+02,1.176467e+01 L7.463103e+02,7.937511e+00 L7.548824e+02,5.170695e+00 L7.634545e+02,3.600000e+00 L7.720267e+02,3.897603e+00 L7.805988e+02,5.187952e+00 L7.891709e+02,7.939000e+00 L7.977430e+02,1.177236e+01 L8.063152e+02,1.710991e+01 L8.148873e+02,2.354074e+01 L8.234594e+02,3.091920e+01 L8.320315e+02,3.935715e+01 L8.406036e+02,4.850126e+01 L8.491758e+02,5.813852e+01 L8.577479e+02,6.857974e+01 L8.663200e+02,7.916387e+01 "/> <path vector-effect="non-scaling-stroke" d="M1.768000e+01,9.004901e+01 L2.625212e+01,1.008779e+02 L3.482424e+01,1.115361e+02 L4.339636e+01,1.218554e+02 L5.196848e+01,1.316730e+02 L6.054061e+01,1.408343e+02 L6.911273e+01,1.491946e+02 L7.768485e+01,1.566222e+02 L8.625697e+01,1.629998e+02 L9.482909e+01,1.682270e+02 L1.034012e+02,1.722213e+02 L1.119733e+02,1.749197e+02 L1.205455e+02,1.762796e+02 L1.291176e+02,1.762796e+02 L1.376897e+02,1.749197e+02 L1.462618e+02,1.722213e+02 L1.548339e+02,1.682270e+02 L1.634061e+02,1.629998e+02 L1.719782e+02,1.566222e+02 L1.805503e+02,1.491946e+02 L1.891224e+02,1.408343e+02 L1.976945e+02,1.316730e+02 L2.062667e+02,1.218553e+02 L2.148388e+02,1.115361e+02 L2.234109e+02,1.008779e+02 L2.319830e+02,9.004900e+01 L2.405552e+02,7.922008e+01 L2.491273e+02,6.856193e+01 L2.576994e+02,5.824265e+01 L2.662715e+02,4.842497e+01 L2.748436e+02,3.926375e+01 L2.834158e+02,3.090342e+01 L2.919879e+02,2.347585e+01 L3.005600e+02,1.709818e+01 L3.091321e+02,1.187100e+01 L3.177042e+02,7.876722e+00 L3.262764e+02,5.178358e+00 L3.348485e+02,3.818459e+00 L3.434206e+02,3.818459e+00 L3.519927e+02,5.178368e+00 L3.605648e+02,7.876737e+00 L3.691370e+02,1.187101e+01 L3.777091e+02,1.709821e+01 L3.862812e+02,2.347588e+01 L3.948533e+02,3.0903

       100% [========================================] (5095 steps/s) [step=799] [loss+=0.0109] [~loss+=0.0666] [~loss=0.0666]                 
